<a href="https://colab.research.google.com/github/sayansays21/Pyspark-/blob/master/simulating%20by%20Monte%20carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
########## Run this code snippet when running for the first time and don't repeat it in future (else it will keep on downloading the same stuffs again and again and
########## result in redundant usage of memory)

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
import findspark

os.environ["JAVA_HOME"]   = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]  = "/content/spark-2.4.4-bin-hadoop2.7"
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import random
import bisect
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from datetime import date
spark                      = SparkSession.builder.master("local[*]").getOrCreate()

In [42]:
pathname = '/content/Project3.csv'
df = spark.read.csv(pathname,header=True,inferSchema=True,sep=",")
df.show(3,truncate=False)


+---+--------------------+-------+-----+-------------+
|_c0|Customer_ID         |Premium|Claim|Current_state|
+---+--------------------+-------+-----+-------------+
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|0            |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|0            |
|2  |ZtdvlVmRJAwbBAfxYpfI|1508   |22967|0            |
+---+--------------------+-------+-----+-------------+
only showing top 3 rows



In [0]:
class Simulate(object):
  def __init__(self):
    '''
      Static proability  matrices provided
    '''
    self.prob_matrix = np.array([
                                [0.95,0.05],[0.99,0.01]
                      ])
   
    self.state     = [0,1]
  def get_vector(self,key):
    '''
      Getting cumulative probability distribution vector corresponding to the current state
    '''
    return np.cumsum(dict(zip(self.state,[self.prob_matrix[i] for i in range(self.prob_matrix.shape[0])]))[key])
  
  def generate_next_state(self,key):
    '''
      Simulating next state
    '''
    r = random.random()
    return self.state[bisect.bisect(self.get_vector(key),r)]
  def generate_entire_path(self,key):
    '''
      Simulates paths with respect to :- simulations and time period
    '''
    path_info = []
    for i in range(5):
      if i == 0:
        c_state = 0
        n_state = key
      else:
        c_state = n_state
        n_state  = self.generate_next_state(c_state)
      path_info.append([i+1,c_state,n_state])
    return path_info

In [0]:
inst = Simulate()
path_update = udf(inst.generate_entire_path,ArrayType(ArrayType(IntegerType())))

strt frm here

In [45]:
df = df.withColumn('outputs',path_update(df['Current_state']))
df = df.cache() ###### This is needed because of the lazy operation nature of pyspark module
df.show(10)

+---+--------------------+-------+-----+-------------+--------------------+
|_c0|         Customer_ID|Premium|Claim|Current_state|             outputs|
+---+--------------------+-------+-----+-------------+--------------------+
|  0|hobVdDEmlEEzUcvndULm|   2886|16975|            0|[[1, 0, 0], [2, 0...|
|  1|fxBjzsDjwwMKeAFbxsYi|   2517|15191|            0|[[1, 0, 0], [2, 0...|
|  2|ZtdvlVmRJAwbBAfxYpfI|   1508|22967|            0|[[1, 0, 0], [2, 0...|
|  3|dxQyxWcPzCjDoOZohZcJ|   1275|23921|            0|[[1, 0, 0], [2, 0...|
|  4|bGwmYRjIhyRSQxPDnBOA|   2775|15284|            0|[[1, 0, 0], [2, 0...|
|  5|fssQOSIlTBGVHWLikhQT|   2282|28780|            0|[[1, 0, 0], [2, 0...|
|  6|IMPRVjYMVcxxibWViEvo|   1096|17064|            0|[[1, 0, 0], [2, 0...|
|  7|AJNTXslAYzcMxzyQAZVp|   1249|17738|            0|[[1, 0, 0], [2, 0...|
|  8|AgmxXXcgezEcTlYMOWPL|   2000|12750|            0|[[1, 0, 0], [2, 0...|
|  9|PIilceQcIaCXsSTWNzIV|   1615|22064|            0|[[1, 0, 0], [2, 0...|
+---+-------

In [46]:
df = df.withColumn('Explode',explode('outputs'))
df = df.drop('outputs')
df.show(10,truncate=False)

+---+--------------------+-------+-----+-------------+---------+
|_c0|Customer_ID         |Premium|Claim|Current_state|Explode  |
+---+--------------------+-------+-----+-------------+---------+
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|0            |[1, 0, 0]|
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|0            |[2, 0, 0]|
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|0            |[3, 0, 0]|
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|0            |[4, 0, 0]|
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|0            |[5, 0, 0]|
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|0            |[1, 0, 0]|
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|0            |[2, 0, 1]|
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|0            |[3, 1, 0]|
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|0            |[4, 0, 0]|
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|0            |[5, 0, 0]|
+---+--------------------+-------+-----+-------------+---------+
only showing top 10 rows



In [47]:
df=df.withColumn("Months",col("Explode").getItem(0))
df=df.withColumn("Present_state",col("Explode").getItem(1))
df=df.withColumn("Next_State",col("Explode").getItem(2))
df=df.drop('Current_state','Explode')
df=df.cache()
df.show(10,truncate=False)

+---+--------------------+-------+-----+------+-------------+----------+
|_c0|Customer_ID         |Premium|Claim|Months|Present_state|Next_State|
+---+--------------------+-------+-----+------+-------------+----------+
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|1     |0            |0         |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|2     |0            |0         |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|3     |0            |0         |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|4     |0            |0         |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|5     |0            |0         |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|1     |0            |0         |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|2     |0            |1         |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|3     |1            |0         |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|4     |0            |0         |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|5     |0            |0         |
+---+--------------------+-------+-----+------+----

In [0]:
class Final(object):
  def profit_or_loss(self,present_s,next_s,premium,claim):
    if (present_s==0 or present_s==1) and next_s==0:
      return premium
    else:
      return (-claim)


In [0]:
final=Final()
gain=udf(final.profit_or_loss,IntegerType())


In [50]:
df=df.withColumn("Gain",gain(df["Present_state"],df["Next_state"],df["Premium"],df["Claim"]))
df=df.cache()
df.show(10,truncate=False)

+---+--------------------+-------+-----+------+-------------+----------+------+
|_c0|Customer_ID         |Premium|Claim|Months|Present_state|Next_State|Gain  |
+---+--------------------+-------+-----+------+-------------+----------+------+
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|1     |0            |0         |2886  |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|2     |0            |0         |2886  |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|3     |0            |0         |2886  |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|4     |0            |0         |2886  |
|0  |hobVdDEmlEEzUcvndULm|2886   |16975|5     |0            |0         |2886  |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|1     |0            |0         |2517  |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|2     |0            |1         |-15191|
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|3     |1            |0         |2517  |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|4     |0            |0         |2517  |
|1  |fxBjzsDjwwMKeAFbxsYi|2517   |15191|

In [72]:
print("profit or loss per customer")
df1=df.groupBy('Customer_ID').agg({'Gain':'sum'}).select("Customer_ID",col("sum(Gain)").alias("Gain or loss per customer"),)
df1.show(10,truncate=False)
print("total profit of the company is",df1.groupBy().sum().collect()[0][0])

profit or loss per customer
+--------------------+-------------------------+
|Customer_ID         |Gain or loss per customer|
+--------------------+-------------------------+
|lMbIDcwCgbDMlWXJctsU|6115                     |
|IzbjgqblIZgSxKdUToGC|10525                    |
|tTJrdsuDsEelbRRjCVqW|6315                     |
|sfKuMbNhICeAkBHYNlsC|7270                     |
|gwcIZtJJoBuzbsaWyDDS|5875                     |
|XLxKDzHRYdWWPoutgsdx|11060                    |
|FFImXPYaynReGdCIEwZT|8110                     |
|FAJqzuJvVYbVeVGWSabW|13665                    |
|vjTlUlqLILZBTDmZLhKo|5175                     |
|tVFfBWwtAbLNrKbOkoxf|-8364                    |
+--------------------+-------------------------+
only showing top 10 rows

total profit of the company is 426692
